### Import necessary libraries

In [91]:
import numpy as np
import pandas as pd 
import json
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
from bs4 import BeautifulSoup
from urllib.request import urlopen
import geocoder

### Scrape table from website

In [83]:
scrape_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

soup = BeautifulSoup(urlopen(scrape_url))
toronto_data = []

for tr in soup.find_all('table')[0].find_all('tr')[1:]:
    tds = tr.find_all('td')
    postal_code = tds[0].text
    borough = tds[1].text
    nhood = tds[2].text
    if (borough != 'Not assigned\n'):
        if nhood == 'Not assigned\n':
            nhood = borough
        toronto_data.append([postal_code, borough, nhood])

### Create dataframe and clean data

In [84]:
df_toronto = pd.DataFrame(toronto_data, columns=['PostalCode', 'Borough', 'Neighborhood'])
for col in df_toronto.columns:
    df_toronto[col] = df_toronto[col].str.strip('\n')


In [87]:
df_toronto.shape

(103, 3)

### Load in coordinates data and join to existing df

In [132]:
!wget -q -O 'toronto_coords.csv' https://cocl.us/Geospatial_data
with open('toronto_coords.csv') as data:
    toronto_coords = pd.read_csv(data)
    
df_toronto = df_toronto.join(toronto_coords.set_index('Postal Code'), on='PostalCode')

In [133]:
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
